In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import matplotlib.patches as patches
import jax.numpy as jnp # this is a thin wrapper to NumPy within JAX
from jax import grad, hessian
from scipy.optimize import minimize
from scipy.optimize import Bounds

mpl.rcParams['lines.linewidth'] = 2
mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18) 
mpl.rc('axes', labelsize=18) 
mpl.rc('font', size=18) 
plt.rcParams.update({
    'text.usetex': True,
    'font.family': 'serif',
})

# Problem: constrained rosenbrock
$\min_x~f(x) = 100\times (x_2 - x_1^2)^2 + (1 - x_1)^2$

subject to

$c_1(x): x_1 + 2x_2 \leq 1$

$c_2(x): x_1^2 + x_2 \leq 1$

$c_3(x): x_1^2 - x_2 \leq 1$

$c_4(x): 2x_1 + x_2 = 1$

$0 \leq x_1 \leq 1$

$0.5 \leq x_2 \leq 2.0$

## Define objective function, constraints, and variable bounds

 - Notice that inequality constraints are redfined to comply with $c_i(x) \geq 0$
 - Notice that bound constraints are separately defined using the `bounds` oobject

In [2]:
def f(x):
    """The Rosenbrock function"""
    return 100.0 * (x[1] - x[0]**2.0)**2.0 + (1-x[0])**2.0

c1 = lambda x: 1 - x[0]    - 2*x[1]
c2 = lambda x: 1 - x[0]**2 - x[1]
c3 = lambda x: 1 - x[0]**2 + x[1]
c4 = lambda x: 2 * x[0]    + x[1] - 1 # equality constraint

bounds = Bounds([0, -0.5], [1.0, 2.0])

# Define constraints for SciPy minimize

This is where we differntiate the inequality from equality constraints

In [3]:
ineq_con1 = {'type': 'ineq',
             'fun': c1,
             'jac': grad(c1)}
ineq_con2 = {'type': 'ineq',
             'fun': c2,
             'jac': grad(c2)}
ineq_con3 = {'type': 'ineq',
             'fun': c3,
             'jac': grad(c3)}
eq_con1   = {'type': 'eq',
             'fun': c4,
             'jac': grad(c4)}

In [8]:
x = np.array([1., 1.])
constraint_array = np.array([c1(x), c2(x), c3(x), c4(x)])
print(f'cons value {constraint_array}')
constraint_violation = np.array([max(0, -c1(x))**2, max(0, -c2(x))**2, max(0, -c3(x))**2, c4(x)**2])
print(f'cons violation {constraint_violation}')
mcv = np.linalg.norm(constraint_violation, np.inf)
print(f'MCV {mcv}')

cons value [-2. -1.  1.  2.]
cons violation [4. 1. 0. 4.]
MCV 4.0


# Define a `callback` function to store optimization histories

In [ ]:
def callback(x):
    xx.append(x) # iterate xk
    fx.append(f(x)) # function value
    c1x.append(ineq_con1['fun'](x)) # constraint evaluation for c1 only

    print(f'xk {x}, fk {f(x)}, c1 {c1(x)} ')

# Finally, solve the constrained optimization problem

In [ ]:
x0 = np.array([1.0, 2.0])

xx = [] 
fx = []
c1x = [] 

res = minimize(f, x0, method='SLSQP',  jac=grad(f),
               constraints=[ineq_con1, ineq_con2, ineq_con3, eq_con1], 
               options={'ftol': 1e-9, 'disp': True},
               bounds=bounds, callback=callback)

# Penalty method
## Define the objective and constraint functions

In [ ]:
branin = lambda x: jnp.asarray( (x[...,1] - 5.1/4/jnp.pi**2 * x[...,0]**2 + 5/jnp.pi * x[...,0] - 6)**2 + 10*(1 - 1/8/jnp.pi)*jnp.cos(x[...,0]) + 10 )
c1 = 
c2 = 
c3 = 
c4 = 
c5 = 
c6 = 
c7 = 

In [ ]:
def linesearch_loop(x0, tauk, func, bounds, disp_iter=False):
    r"""
    This is the full loop of line search that you implemented for HWs 2&3
    The arguments and returning variables are only suggestions (what I used)
    """

    print('----------------------------------')
    return xk, np.array(ginf_fr)

In [ ]:
mu0 = 0.001
eta = 0.5
rho = 2.
tau0= 1.
x0 = np.array([6., 14.])
xk_array = [x0]
gfp_array = []
bounds_branin = np.array([[-5., 0], [10., 15.]])
project_x = lambda x, bounds=bounds_branin: np.maximum(np.minimum(x, bounds[1]), bounds[0])

tauk = tau0
muk  = mu0
while tauk > 1e-4 or muk < 10:
    f_penalized = lambda x: branin(x) + 0.5 * muk * c3(x)**2
    xk, ginf_fr = linesearch_loop(x0, tauk, f_penalized, bounds_branin, disp_iter=True)
    x0 = xk # starting points for next iteration
    muk = muk * rho
    tauk= tauk* eta
    xk_array.append(xk)
    gfp_array.append(ginf_fr)

In [ ]:
# plot branin
m = 40
xv, yv = np.meshgrid(np.linspace(-5,10, m), np.linspace(0,15,m))
x = np.column_stack((xv.reshape(-1,1), yv.reshape(-1,1)))
xk_ = np.array(xk_array)

fig = plt.figure(figsize=(12,8))
ax2 = fig.add_subplot(121)
cp  = ax2.contourf(xv,yv,branin(x).reshape(m,m), levels=25)
cp3 = ax2.contour(xv,yv,c3(x).reshape(m,m), levels=[0.], linewidths=4, colors='g')
ax2.plot([6.], [14.], marker='o', c='k')
ax2.plot(xk_[:,0], xk_[:,1], c='w', marker='.', label='trajectory')
ax2.set_xlabel(r'$x_1$')
ax2.set_ylabel(r'$x_2$')
ax2.legend()

gfp = np.concatenate(gfp_array)
ax1 = fig.add_subplot(122)
ax1.semilogy(np.arange(gfp.shape[0]), gfp, marker='.', lw=2)
ax1.set_xlabel('accumultaed iterations')
ax1.set_ylabel(r'$\| \nabla \hat{f}(\mathbf{x}_k; \mu_k)\|$')
plt.tight_layout()